# Forecasting methods (I/II)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import os

from main.utils.utils_methods import *
from main.utils.utils import *
import statsmodels.api as sm
from pmdarima.arima import auto_arima

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from main.module.mlp_multioutput import mlp_multioutput


%load_ext autoreload
%autoreload 2

We will simulate a time series from the following ARIMA process:
$$
y_t = 0.75 y_{t-1} - 0.25 y_{t-2} + 0.65 \varepsilon_{t-1} + 0.35 \varepsilon_{t-2} + \varepsilon_t
$$

In [ ]:
np.random.seed(12345)

# ARMA parameters
arparams = np.array([.75, -.25])
maparams = np.array([.65, .35])
ar_term = np.r_[1, -arparams] # add zero-lag and negate
ma_term = np.r_[1, maparams] # add zero-lag
arma_process = sm.tsa.ArmaProcess(ar_term, ma_term)

y = arma_process.generate_sample(1000)
plt.plot(y)

Fit an ARMA model to the simulated time series using auto_arima. Do you recover the true parameters? To obtain details about your model fit, you can use the following functions: summary(), arparams(), and maparams().

In [ ]:
# ??


Change the default parameters of the auto_arima function to get faster results. Eploit the fact that you know the "true" model (see above).

In [ ]:

# ??


# Neural network forecast

In the following, we will consider two neural network architectures for forecasting. Your task is to play with all the hyperparameters to obtain the best out-of-sample forecasts, i.e. on the test set.

Some important hyperparameters include: n_simul (size of the dataset), LAG (the number of lagged values), LATENT_DIM (the number of units in the layer), BATCH_SIZE (number of samples per mini-batch), EPOCHS (the number of epochs), the optimizer and the early stop strategy.

The dataset you will use is a time series simulated from a non-linear time series model. See below. 

In [ ]:
n_simul = 1000
n_burn = 100
n = n_simul + n_burn
noise = np.random.normal(size = n)

y = np.zeros(n)
y[0] = 0; y[1] = 0
for t in range(2, n):
    y[t] = 0.3 * y[t - 1] + 0.6 * y[t - 2] + (0.1 - 0.9 * y[t - 1] + 0.8 * y[t - 2]) * (1/( 1 + np.exp(- 10 * y[t - 1]) )) + noise[t]

data = pd.DataFrame(y[n_burn:], columns = ["series"])
plt.plot(data)

Choose which loss function you want to experiment with. It is used later in the code to fit and evaluate a neural network model.

In [ ]:
# Loss function to be used to optimize the model parameters
loss_fct = 'mse' # 'mae'
# Accuracy measure to be used to evaluate test predictions.
accuracy_measure = mse # mae # mape # smape

In [ ]:
# The forecast horizon
HORIZON = 3

# The number of lagged values.
LAG = 4

# Data split
n = len(data)
n_train = int(0.6 * n)
n_valid = int(0.2 * n)
n_learn = n_train + n_valid

train = data[:n_train]
valid = data[n_train:n_learn]
test = data[n_learn:n]

# From time series to input-output data (also called time series embedding)
train_inputs, valid_inputs, test_inputs, \
    X_train, y_train, X_valid, y_valid, \
        X_test, y_test = embed_data(train, valid, test, HORIZON, LAG, freq = None, variable = 'series')


Read and try to understand the function *mlp_multioutput*. What kind of neural network architecture does it return?

In [ ]:
#########################
file_header = "model_" + "mlp_multioutput"
verbose = 0

optimizer_adam = keras.optimizers.Adam(learning_rate=0.01) 
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience= 100)

LATENT_DIM = 5   # number of units in the RNN layer
BATCH_SIZE = 32  # number of samples per mini-batch
EPOCHS = 100      # maximum number of times the training algorithm will cycle through all samples
loss = loss_fct

best_val = ModelCheckpoint('../work/' + file_header + '_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)
#########################

model_mlp_multioutput, history_mlp_multioutput = mlp_multioutput(X_train, y_train, X_valid, y_valid, 
                        LATENT_DIM = LATENT_DIM, 
                        BATCH_SIZE = BATCH_SIZE, 
                        EPOCHS = EPOCHS, 
                        LAG = LAG, 
                        HORIZON = HORIZON, 
                        loss = loss, 
                        optimizer = optimizer_adam,
                        earlystop = earlystop, 
                        best_val = best_val,
                        verbose=verbose)
plot_learning_curves(history_mlp_multioutput)

best_epoch = np.argmin(np.array(history_mlp_multioutput.history['val_loss']))+1
filepath = '../work/' + file_header + '_{:02d}.h5'
model_mlp_multioutput.load_weights(filepath.format(best_epoch))

In [ ]:
 #########################
file_header = "model_" + "mlp_recursive"
verbose = 0

optimizer_adam = keras.optimizers.Adam(learning_rate=0.01) 
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience= 100)

LATENT_DIM = 5   # number of units in the RNN layer
BATCH_SIZE = 32  # number of samples per mini-batch
EPOCHS = 100      # maximum number of times the training algorithm will cycle through all samples
loss = loss_fct

best_val = ModelCheckpoint('../work/' + file_header + '_{epoch:02d}.h5', save_best_only=True, mode='min', period=1)
#########################
 
 _, _, _, X_train_onestep, y_train_onestep, X_valid_onestep, y_valid_onestep, _, _ = embed_data(train, valid, test, 1, LAG, freq = None, variable = 'series')
model_mlp_recursive, history_mlp_recursive = mlp_multioutput(X_train_onestep, y_train_onestep, X_valid_onestep, y_valid_onestep, 
                        LATENT_DIM = LATENT_DIM, 
                        BATCH_SIZE = BATCH_SIZE, 
                        EPOCHS = EPOCHS, 
                        LAG = LAG, 
                        HORIZON = 1, 
                        loss = loss, 
                        optimizer = optimizer_adam,
                        earlystop = earlystop, 
                        best_val = best_val,
                        verbose=verbose)
plot_learning_curves(history_mlp_recursive)

best_epoch = np.argmin(np.array(history_mlp_recursive.history['val_loss']))+1
filepath = '../work/' + file_header + '_{:02d}.h5'
model_mlp_recursive.load_weights(filepath.format(best_epoch))


In [ ]:
#len(X_test.values[:, -1])
predictions_naive = np.tile(X_test.values[:, -1], (HORIZON, 1) ).T
predictions_naive = pd.DataFrame(predictions_naive, columns=['t+'+str(t) for t in range(1, HORIZON+1)])
print(predictions_naive)


In [ ]:
#
predictions_mlp_multioutput = model_mlp_multioutput.predict(X_test)
predictions_mlp_multioutput = pd.DataFrame(predictions_mlp_multioutput, columns=['t+'+str(t) for t in range(1, HORIZON+1)])

#
for h in range(HORIZON):
    pred = model_mlp_recursive.predict(X_test)
    X_test = pd.DataFrame(np.hstack( (np.delete(X_test.to_numpy(), 0, 1), pred) ), index = X_test.index, columns =X_test.columns)
    if h > 0:
        predictions_mlp_recursive = np.hstack( (predictions_mlp_recursive, pred) )
    else:
        predictions_mlp_recursive = pred
predictions_mlp_recursive = pd.DataFrame(predictions_mlp_recursive, columns=['t+'+str(t) for t in range(1, HORIZON+1)])

predictions_combination = (predictions_mlp_multioutput + predictions_mlp_recursive)/2

print(predictions_mlp_multioutput)
print(predictions_mlp_recursive)

In [ ]:
true_values = pd.DataFrame(test_inputs["target"], columns=['t+'+str(t) for t in range(1, HORIZON+1)])

results_naive = list()
results_mlp_multioutput = list()
results_mlp_recursive = list()
results_combination = list()


for h in range(1, HORIZON+1):
    time_horizon = 't+'+ str(h)
    results_naive.append(accuracy_measure(true_values[time_horizon], predictions_naive[time_horizon]))
    results_mlp_multioutput.append(accuracy_measure(true_values[time_horizon], predictions_mlp_multioutput[time_horizon]))
    results_mlp_recursive.append(accuracy_measure(true_values[time_horizon], predictions_mlp_recursive[time_horizon]))
    results_combination.append(accuracy_measure(true_values[time_horizon], predictions_combination[time_horizon]))


print(np.mean(results_naive))
print(np.mean(results_mlp_multioutput))
print(np.mean(results_mlp_recursive))
print(np.mean(results_combination))


#print( mape(true_values.to_numpy().T.ravel(), predictions_mlp.to_numpy().T.ravel()) )



Run the four previous forecasting methods on one time series of the Kaggle dataset, and compare their performances. Do not forget to apply transformation to make the series stationary before running neural network models.